In [1]:
import json
import csv

import pandas as pd
import numpy as np

In [2]:
region_name = "Томск"
region_data_file = "tomskaia-oblast.geojson"  # from https://dtp-stat.ru/opendata/

In [3]:
def get_dtp_by_region():
    def check_region(item):
        return item["properties"]["region"] == region_name

    with open(region_data_file, "rt") as f:
        dtp = json.load(f)

    res = list(filter(check_region, dtp["features"]))
    print(f'всего ДТП в регионе "{region_name}": {len(res)}')
    return res
dtp_tomsk = get_dtp_by_region()

всего ДТП в регионе "Томск": 2318


In [17]:
def feature_extractor(it):
    res = {
        "id": it["properties"]["id"],
        "datetime": it["properties"]["datetime"],
        "lat": it["properties"]["point"]["lat"],
        "lon": it["properties"]["point"]["long"],
        "severity": it["properties"]["severity"],
        "address": it["properties"]["address"],
        "dead_count": it["properties"]["dead_count"],
        "injured_count": it["properties"]["injured_count"],
    }
    return res
df_dtp = pd.DataFrame(map(feature_extractor, dtp_tomsk))
df_dtp.datetime = df_dtp.datetime.astype(np.datetime64)
df_dtp.head()

,id,datetime,lat,lon,severity,address,dead_count,injured_count
0,2528452,2020-12-20 23:02:00,56.503054,84.958584,Легкий,"г Томск, ул Дальне-Ключевская, 25Б",0,1
1,2503256,2020-10-07 20:15:00,56.576648,85.105934,С погибшими,"г Томск, Кузовлево-Светлый, 6 км",1,0
2,2539023,2021-01-31 08:20:00,56.496890,84.968401,Легкий,"г Томск, ул Яковлева, 9",0,1
3,2539024,2021-01-30 14:30:00,56.462004,84.989151,Легкий,None,0,1
4,2539025,2021-01-29 21:36:00,56.480713,85.033740,Тяжёлый,"г Томск, ул Писемского, 8",0,1


In [5]:
repairs = []
with open("00_fragments.csv", "rt") as f:
    reader = csv.reader(f)
    header = next(reader)
    print("Header columns:", header)
    for row in reader:
        repairs.append(dict(zip(header,row)))
df_repairs = pd.DataFrame(repairs)
df_repairs

Header columns: ['Год работ', 'fragment_header', 'fragment_title', 'comments', 'id', 'Статус', 'Срок выполнения работ на объекте', 'Тип работ']


,Год работ,fragment_header,fragment_title,comments,id,Статус,Срок выполнения работ на объекте,Тип работ
0,2021,Ремонт,пр.Коммунистического от ул.Свердлова до ул.Лео...,,e8809a70-2216-4bee-922b-154d73ae493f,Запланирован,25.04.2021 ‒ 01.09.2021,Ремонт
1,2017,Ремонт покрытия проезжей части,Автодорога «ул.Кирова»,,70ee27cb-ab73-4efb-bcc2-93c0b5a4b85c,Завершен,01.06.2017 ‒ 31.07.2017,Ремонт покрытия проезжей части
2,2020,Ремонт,ул.Крупской: от просп.Коммунистического до ул....,,4a52c0b2-7805-4bf1-b9b7-d090263c28de,Завершен,25.04.2020 ‒ 31.08.2020,Ремонт
3,2018,Ремонт покрытия проезжей части,ул. Войкова,,56402bec-90dc-4184-bbda-07e4a7191180,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части
4,2018,Ремонт покрытия проезжей части,ул. Набережная р. Ушайки,,25d81872-36a6-4066-8132-58f14f969e50,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части
...,...,...,...,...,...,...,...,...
161,2019,Ремонт,автодорога по просп.Коммунистическому-кольцо н...,,19d7af5a-54eb-4169-b208-4e836da75c64,Завершен,10.05.2019 ‒ 30.08.2019,Ремонт
162,2019,Ремонт,ул. Смирнова от ул.Смирнова 1 до ул.Кольцевой ...,,8d86833b-14c7-47a6-9522-9f640bbce034,Завершен,15.05.2019 ‒ 08.10.2019,Ремонт
163,2017,Ремонт покрытия проезжей части,ул. 1-ая Ново-Деповская,,60cee825-414b-4c52-b632-eb863e351243,Завершен,18.04.2017 ‒ 31.08.2017,Ремонт покрытия проезжей части
164,2017,Ремонт покрытия проезжей части,Чекистский тракт,,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,Завершен,18.04.2017 ‒ 05.10.2017,Ремонт покрытия проезжей части


In [11]:
with open("01_dtp_to_fragments.json", "rt") as f:
    dtp_to_repair = json.load(f)
df_dtp_to_repair = pd.DataFrame(dtp_to_repair)

In [8]:
pd.DataFrame(dtp_to_repair)

,dtp_id,fragment
0,2528452,None
1,2503256,568621a0-d426-4298-ada8-bd7eab8c0aa3
2,2539023,5c08e4b8-1251-44fe-aec8-129cfc5f1eb4
3,2539024,037f9264-cc4f-4cd8-aef2-585e44cef92c
4,2539025,None
...,...,...
2310,2281696,None
2311,2281710,d7a81235-4765-4f4e-b6e7-736e5f0a64cf
2312,2281683,94e5149a-593a-4cec-baf6-ea0c12864a39
2313,2281704,e726f474-cc35-4895-89db-f0ec92666d5b


In [20]:
# for every repair match all related DTPs
df_merged = df_repairs\
    .merge(df_dtp_to_repair, left_on="id", right_on="fragment")\
    .merge(df_dtp, left_on="dtp_id", right_on="id", suffixes=("_repair", "_dtp"))

In [30]:
df_merged["name"] = df_merged.fragment_header + " " + df_merged.fragment_title + ", работы проводились " \
                    + df_merged["Срок выполнения работ на объекте"]
df_merged.severity = df_merged.severity.astype("category")
df_merged.severity.cat.set_categories(["Легкий", "Тяжёлый", "С погибшими"], inplace=True)  # sorting order
df_merged["year"] = df_merged.datetime.dt.year

In [38]:
df_res = df_merged.groupby(["year", "severity", "name"]).agg(count=("dtp_id", "size"))\
    .unstack().fillna(0).droplevel(0, axis=1)
df_res

name              Ремонт Выполнение работ по ремонту автомобильной дороги Подъезд к с. Дзержинское на участке км 0+000 - км 2+413 в Томском районе Томской области, работы проводились 18.11.2019 ‒ 01.10.2020  \
year severity                                                                                                                                                                                                    
2015 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      0                                                                                                                                              
     С погибшими                                                  0                                                                                                                                              
2016 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      1                                                                                                                                              
     С погибшими                                                  0                                                                                                                                              
2017 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      0                                                                                                                                              
     С погибшими                                                  0                                                                                                                                              
2018 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      0                                                                                                                                              
     С погибшими                                                  0                                                                                                                                              
2019 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      0                                                                                                                                              
     С погибшими                                                  0                                                                                                                                              
2020 Легкий                                                       0                                                                                                                                              
     Тяжёлый                                                      0                                                                                                                                              
     С пог

In [47]:
df_res.to_excel("results.xlsx")

In [41]:
df_res.loc[:, 'Ремонт покрытия проезжей части пр. Кирова, работы проводились 10.04.2017 ‒ 31.08.2017']

year  severity   
2015  Легкий          3
      Тяжёлый         5
      С погибшими     0
2016  Легкий         10
      Тяжёлый         5
      С погибшими     0
2017  Легкий          3
      Тяжёлый         5
      С погибшими     0
2018  Легкий          4
      Тяжёлый         7
      С погибшими     1
2019  Легкий         10
      Тяжёлый         4
      С погибшими     0
2020  Легкий          7
      Тяжёлый         3
      С погибшими     0
2021  Легкий          1
      Тяжёлый         1
      С погибшими     0
Name: Ремонт покрытия проезжей части пр. Кирова, работы проводились 10.04.2017 ‒ 31.08.2017, dtype: int64

In [44]:
df_res.columns[df_res.columns.str.contains("Учебная")]

Index(['Ремонт покрытия проезжей части ул. Учебная, работы проводились 02.05.2017 ‒ 31.08.2017'], dtype='object', name='name')

In [45]:
df_res.loc[:, 'Ремонт покрытия проезжей части ул. Учебная, работы проводились 02.05.2017 ‒ 31.08.2017']

year  severity   
2015  Легкий         2
      Тяжёлый        3
      С погибшими    0
2016  Легкий         1
      Тяжёлый        2
      С погибшими    0
2017  Легкий         4
      Тяжёлый        0
      С погибшими    0
2018  Легкий         4
      Тяжёлый        4
      С погибшими    0
2019  Легкий         4
      Тяжёлый        3
      С погибшими    0
2020  Легкий         0
      Тяжёлый        2
      С погибшими    0
2021  Легкий         0
      Тяжёлый        0
      С погибшими    0
Name: Ремонт покрытия проезжей части ул. Учебная, работы проводились 02.05.2017 ‒ 31.08.2017, dtype: int64